In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
sys.path.append('../')

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
from IPython.display import HTML, display
from sklearn import metrics
import traceback 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import featuretools as ft

MERED = "merged"
PATH = "../../../data/home-credit-default-risk/"
#PATH = "../../data/home-credit-default-risk/"

/home/max/anaconda3/envs/fastai-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/max/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
ls {PATH}

application_test.csv                merged/
application_train.csv               models/
bureau_balance.csv                  POS_CASH_balance.csv
bureau.csv                          previous_application.csv
credit_card_balance.csv             sample_submission.csv
HomeCredit_columns_description.csv  tmp/
installments_payments.csv


# Bureau + bureau balance

In [3]:
def load_sample(file_name, sk_id_currs):    
    table = pd.read_csv(f'{PATH}{file_name}.csv',low_memory=False)    
    table = table[table.SK_ID_CURR.isin(sk_id_currs)]
    print(file_name, len(table))
    return table

TODO: here need concat application_train + application_test; with is_test collumn

In [4]:
file_names = ['application_train']
tables = [pd.read_csv(f'{PATH}{file_name}.csv',low_memory=False, nrows=10000) for file_name in file_names]
train = tables[0]

In [14]:
bureau = load_sample('bureau', train.SK_ID_CURR)

bureau 46974


In [15]:
file_names = ['bureau_balance']
tables = [pd.read_csv(f'{PATH}{file_name}.csv',low_memory=False) for file_name in file_names]
bureau_balance = tables[0][tables[0].SK_ID_BUREAU.isin(bureau.SK_ID_BUREAU)]

In [16]:
len(train), len(bureau), len(bureau_balance)

(10000, 46974, 474218)

In [8]:
bureau_balance = bureau_balance.reset_index()

In [9]:
entities = {
    'train' : (train, "SK_ID_CURR"),
    'bureau' : (bureau, "SK_ID_BUREAU"),
    'bureau_balance' : (bureau_balance, 'index')
}

relationships = [
    ('train', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
    ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU')
]

In [10]:
feature_matrix, feature_defs = ft.dfs(
    entities=entities,
    relationships = relationships,
    target_entity="train",
    agg_primitives=["mean", "sum", "mode"])

In [11]:
len(feature_matrix)

10000

In [12]:
feature_defs

[<Feature: TARGET>,
 <Feature: NAME_CONTRACT_TYPE>,
 <Feature: CODE_GENDER>,
 <Feature: FLAG_OWN_CAR>,
 <Feature: FLAG_OWN_REALTY>,
 <Feature: CNT_CHILDREN>,
 <Feature: AMT_INCOME_TOTAL>,
 <Feature: AMT_CREDIT>,
 <Feature: AMT_ANNUITY>,
 <Feature: AMT_GOODS_PRICE>,
 <Feature: NAME_TYPE_SUITE>,
 <Feature: NAME_INCOME_TYPE>,
 <Feature: NAME_EDUCATION_TYPE>,
 <Feature: NAME_FAMILY_STATUS>,
 <Feature: NAME_HOUSING_TYPE>,
 <Feature: REGION_POPULATION_RELATIVE>,
 <Feature: DAYS_BIRTH>,
 <Feature: DAYS_EMPLOYED>,
 <Feature: DAYS_REGISTRATION>,
 <Feature: DAYS_ID_PUBLISH>,
 <Feature: OWN_CAR_AGE>,
 <Feature: FLAG_MOBIL>,
 <Feature: FLAG_EMP_PHONE>,
 <Feature: FLAG_WORK_PHONE>,
 <Feature: FLAG_CONT_MOBILE>,
 <Feature: FLAG_PHONE>,
 <Feature: FLAG_EMAIL>,
 <Feature: OCCUPATION_TYPE>,
 <Feature: CNT_FAM_MEMBERS>,
 <Feature: REGION_RATING_CLIENT>,
 <Feature: REGION_RATING_CLIENT_W_CITY>,
 <Feature: WEEKDAY_APPR_PROCESS_START>,
 <Feature: HOUR_APPR_PROCESS_START>,
 <Feature: REG_REGION_NOT_LIVE_REG

# All else

In [9]:
def add_table_to_app(app, table):
    entities = {
        'app' : (app, "SK_ID_CURR"),
        'table' : (table, 'index'),
    }
    relationships = [
        ('app', 'SK_ID_CURR', 'table', 'index')
    ]
    
    #TODO: need drop SK_ID_CURR, prev_app, and other indexes
    
    feature_matrix, feature_defs = ft.dfs(
        entities=entities,
        relationships = relationships,
        target_entity="app",
        agg_primitives=["mean", "sum", "mode"])
    
    os.makedirs('{PATH}tmp', exist_ok=True)
    feature_matrix = feature_matrix.reset_index()
    
    feature_matrix.to_feather(f'{PATH}tmp/app-table')        
    
    return feature_matrix, feature_defs

In [19]:
previous_app = load_sample('previous_application', train.SK_ID_CURR)

previous_application 46154


In [20]:
previous_app = previous_app.reset_index()

In [7]:
pos_cash = load_sample('POS_CASH_balance', train.SK_ID_CURR)

POS_CASH_balance 277650


In [10]:
pos_cash = pos_cash.reset_index()

In [8]:
cc_balance = load_sample('credit_card_balance', train.SK_ID_CURR)

credit_card_balance 105628


In [11]:
cc_balance = cc_balance.reset_index()

In [9]:
installments_payments = load_sample('installments_payments', train.SK_ID_CURR)

installments_payments 375092


In [12]:
installments_payments = installments_payments.reset_index()

In [ ]:
entities = {
    'train' : (train, "SK_ID_CURR"),
    'previous_app' : (previous_app, "SK_ID_PREV "),
    'pos_cash' : (pos_cash, 'index'),
    'cc_balance' : (cc_balance, 'index'),
    'installments_payments' : (installments_payments, 'index')
}

relationships = [
    ('train', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
    ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU')
]

In [37]:
feature_matrix, feature_defs = add_table_to_app(train, previous_app)

In [6]:
file_names = ['previous_application', 'POS_CASH_balance', 'credit_card_balance', 'installments_payments']
for file_name in file_names:
    table = load_sample(file_name, train.SK_ID_CURR)
    table = table.reset_index()
    train, feature_defs = add_table_to_app(train, table)

previous_application 46154
POS_CASH_balance 277650
credit_card_balance 105628
installments_payments 375092


In [7]:
df_all = pd.read_feather(f'{PATH}tmp/app-table')

/home/max/.local/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [8]:
df_all.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,MEAN(table.AMT_INSTALMENT),MEAN(table.AMT_PAYMENT),SUM(table.SK_ID_PREV),SUM(table.SK_ID_CURR),SUM(table.NUM_INSTALMENT_VERSION),SUM(table.NUM_INSTALMENT_NUMBER),SUM(table.DAYS_INSTALMENT),SUM(table.DAYS_ENTRY_PAYMENT),SUM(table.AMT_INSTALMENT),SUM(table.AMT_PAYMENT)
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,447329.61,447329.61,NaN,NaN,2.0,6.0,-778.0,-789.0,447329.61,447329.61
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,859.77,859.77,NaN,NaN,0.0,74.0,-783.0,-783.0,859.77,859.77
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
len(train.columns), len(previous_app.columns), len(feature_matrix.columns)

(122, 38, 180)

In [32]:
 with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(DataFrameSummary(feature_matrix).summary())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,MEAN(table.SK_ID_PREV),MEAN(table.SK_ID_CURR),MEAN(table.AMT_ANNUITY),MEAN(table.AMT_APPLICATION),MEAN(table.AMT_CREDIT),MEAN(table.AMT_DOWN_PAYMENT),MEAN(table.AMT_GOODS_PRICE),MEAN(table.HOUR_APPR_PROCESS_START),MEAN(table.NFLAG_LAST_APPL_IN_DAY),MEAN(table.RATE_DOWN_PAYMENT),MEAN(table.RATE_INTEREST_PRIMARY),MEAN(table.RATE_INTEREST_PRIVILEGED),MEAN(table.DAYS_DECISION),MEAN(table.SELLERPLACE_AREA),MEAN(table.CNT_PAYMENT),MEAN(table.DAYS_FIRST_DRAWING),MEAN(table.DAYS_FIRST_DUE),MEAN(table.DAYS_LAST_DUE_1ST_VERSION),MEAN(table.DAYS_LAST_DUE),MEAN(table.DAYS_TERMINATION),MEAN(table.NFLAG_INSURED_ON_APPROVAL),SUM(table.SK_ID_PREV),SUM(table.SK_ID_CURR),SUM(table.AMT_ANNUITY),SUM(table.AMT_APPLICATION),SUM(table.AMT_CREDIT),SUM(table.AMT_DOWN_PAYMENT),SUM(table.AMT_GOODS_PRICE),SUM(table.HOUR_APPR_PROCESS_START),SUM(table.NFLAG_LAST_APPL_IN_DAY),SUM(table.RATE_DOWN_PAYMENT),SUM(table.RATE_INTEREST_PRIMARY),SUM(table.RATE_INTEREST_PRIVILEGED),SUM(table.DAYS_DECISION),SUM(table.SELLERPLACE_AREA),SUM(table.CNT_PAYMENT),SUM(table.DAYS_FIRST_DRAWING),SUM(table.DAYS_FIRST_DUE),SUM(table.DAYS_LAST_DUE_1ST_VERSION),SUM(table.DAYS_LAST_DUE),SUM(table.DAYS_TERMINATION),SUM(table.NFLAG_INSURED_ON_APPROVAL),MODE(table.NAME_CONTRACT_TYPE),MODE(table.WEEKDAY_APPR_PROCESS_START),MODE(table.FLAG_LAST_APPL_PER_CONTRACT),MODE(table.NAME_CASH_LOAN_PURPOSE),MODE(table.NAME_CONTRACT_STATUS),MODE(table.NAME_PAYMENT_TYPE),MODE(table.CODE_REJECT_REASON),MODE(table.NAME_TYPE_SUITE),MODE(table.NAME_CLIENT_TYPE),MODE(table.NAME_GOODS_CATEGORY),MODE(table.NAME_PORTFOLIO),MODE(table.NAME_PRODUCT_TYPE),MODE(table.CHANNEL_TYPE),MODE(table.NAME_SELLER_INDUSTRY),MODE(table.NAME_YIELD_GROUP),MODE(table.PRODUCT_COMBINATION)
count,10000,10000,NaN,NaN,NaN,NaN,10000,10000,10000,10000,9993,NaN,NaN,NaN,NaN,